# Baseline Models

**Purpose**: Establish baseline performance using simple models.

**Why baselines are important**:
- Give us a reference point ("how good is good?")
- Simple models sometimes outperform complex ones on small datasets
- Help us understand if complex models (CNNs) are actually necessary

**Models we'll test**:
1. **Majority Class**: Always predict "Forward" (simplest possible)
2. **Random**: Random guessing (worst case)
3. **Logistic Regression**: Linear classifier
4. **Random Forest**: Non-linear tree-based classifier

**For each model**, we'll test three scenarios:
- A: Original imbalanced data
- B: Original data + class weights
- C: TFI-balanced data

## 1. Setup and Load Data

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import time

# Machine learning models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

# Evaluation metrics
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score
)

# Settings
plt.style.use('default')
%matplotlib inline

# Label names
label_names = {-1: 'Left', 0: 'Forward', 1: 'Right'}

print("✅ Libraries imported successfully")

### Load Temporal Splits (Original)

In [ ]:
# Load original temporal splits (from notebook 01)
data_original = np.load('../data/splits_temporal.npz')

X_train = data_original['X_train']
y_train = data_original['y_train']
X_val = data_original['X_val']
y_val = data_original['y_val']
X_test = data_original['X_test']
y_test = data_original['y_test']

print("Original Temporal Splits:")
print(f"  Train: {len(X_train)} samples - {Counter(y_train)}")
print(f"  Val:   {len(X_val)} samples")
print(f"  Test:  {len(X_test)} samples")

### Load Class Weights

In [ ]:
# Load class weights (from notebook 02)
class_weights = np.load('../data/class_weights.npy', allow_pickle=True).item()

print("Class Weights:")
for k, v in class_weights.items():
    label = [-1, 0, 1][k]  # Map 0→-1, 1→0, 2→1
    print(f"  {label_names[label]:8s}: {v:.3f}")

print("\nThese weights will be used in loss functions to penalize minority class errors more.")

### Load TFI-Balanced Splits

In [ ]:
# Load TFI-balanced splits (from notebook 02)
data_tfi = np.load('../data/splits_temporal_tfi.npz')

X_train_tfi = data_tfi['X_train']
y_train_tfi = data_tfi['y_train']
# Val and test are same as original

print("TFI-Balanced Temporal Splits:")
print(f"  Train: {len(X_train_tfi)} samples - {Counter(y_train_tfi)}")
print(f"  Val:   {len(X_val)} samples (same as original)")
print(f"  Test:  {len(X_test)} samples (same as original)")

print(f"\n✅ Training set increased from {len(X_train)} to {len(X_train_tfi)} samples")
print("   (Added interpolated frames for Left and Right classes)")

### Prepare Data for Models

**Why flatten?** Traditional ML models (not CNNs) expect:
- Input: 1D feature vector (not 2D image)
- We need to convert 64×64 image → 4,096 features

**Normalization**: Scale pixel values from [0, 255] to [0, 1]
- Helps models converge faster
- Prevents features with larger values from dominating

In [ ]:
# Flatten images to 1D vectors
# Original: (samples, 64, 64) → (samples, 4096)
X_train_flat = X_train.reshape(len(X_train), -1)
X_val_flat = X_val.reshape(len(X_val), -1)
X_test_flat = X_test.reshape(len(X_test), -1)

# TFI-balanced
X_train_tfi_flat = X_train_tfi.reshape(len(X_train_tfi), -1)

print("Flattened shapes:")
print(f"  X_train: {X_train.shape} → {X_train_flat.shape}")
print(f"  X_train_tfi: {X_train_tfi.shape} → {X_train_tfi_flat.shape}")

# Normalize to [0, 1]
X_train_flat = X_train_flat / 255.0
X_val_flat = X_val_flat / 255.0
X_test_flat = X_test_flat / 255.0
X_train_tfi_flat = X_train_tfi_flat / 255.0

print("\n✅ Data normalized to [0, 1] range")
print(f"   Min value: {X_train_flat.min():.3f}, Max value: {X_train_flat.max():.3f}")

## 2. Helper Functions for Evaluation

**Purpose**: Create reusable functions to evaluate all models consistently.

In [ ]:
def evaluate_model(y_true, y_pred, model_name="Model"):
    """
    Comprehensive evaluation of model predictions.
    
    Args:
        y_true: True labels
        y_pred: Predicted labels
        model_name: Name for printing
    
    Returns:
        dict with all metrics
    """
    # Calculate metrics
    acc = accuracy_score(y_true, y_pred)
    f1_macro = f1_score(y_true, y_pred, average='macro')  # Average of per-class F1
    f1_per_class = f1_score(y_true, y_pred, average=None, labels=[-1, 0, 1])
    cm = confusion_matrix(y_true, y_pred, labels=[-1, 0, 1])
    
    # Print results
    print(f"\n{'='*60}")
    print(f"{model_name} Results")
    print(f"{'='*60}")
    print(f"Overall Accuracy: {acc:.3f} ({acc*100:.1f}%)")
    print(f"F1-Macro (avg):   {f1_macro:.3f}")
    print("\nPer-class F1 scores:")
    for label, f1 in zip([-1, 0, 1], f1_per_class):
        print(f"  {label_names[label]:8s}: {f1:.3f}")
    
    # Classification report
    print("\nDetailed Classification Report:")
    print(classification_report(y_true, y_pred, 
                                target_names=['Left', 'Forward', 'Right'],
                                labels=[-1, 0, 1]))
    
    return {
        'accuracy': acc,
        'f1_macro': f1_macro,
        'f1_left': f1_per_class[0],
        'f1_forward': f1_per_class[1],
        'f1_right': f1_per_class[2],
        'confusion_matrix': cm
    }


def plot_confusion_matrix(cm, title="Confusion Matrix"):
    """
    Plot confusion matrix as heatmap.
    
    Args:
        cm: Confusion matrix from sklearn
        title: Plot title
    """
    plt.figure(figsize=(8, 6))
    
    # Create heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Left', 'Forward', 'Right'],
                yticklabels=['Left', 'Forward', 'Right'],
                cbar_kws={'label': 'Count'})
    
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Calculate per-class accuracy
    print("Per-class accuracy (diagonal / row sum):")
    for i, label in enumerate([-1, 0, 1]):
        class_acc = cm[i, i] / cm[i, :].sum()
        print(f"  {label_names[label]:8s}: {class_acc:.3f} ({class_acc*100:.1f}%)")


print("✅ Helper functions defined")

## 3. Baseline 1: Majority Class Classifier

**What it does**: Always predicts "Forward" (the most common class)

**Why test this?**
- Simplest possible strategy
- Shows the "do nothing" baseline
- Due to 74% Forward samples, this will get 74% accuracy!
- BUT: 0% accuracy on turns (completely useless for actual driving)

**Key insight**: This shows why overall accuracy is misleading for imbalanced data.

In [ ]:
# Create majority class classifier
# DummyClassifier with strategy='most_frequent' always predicts most common class
majority_clf = DummyClassifier(strategy='most_frequent', random_state=42)
majority_clf.fit(X_train_flat, y_train)

# Predict on test set
y_pred_majority = majority_clf.predict(X_test_flat)

# Evaluate
results_majority = evaluate_model(y_test, y_pred_majority, "Majority Class Baseline")

# Plot confusion matrix
plot_confusion_matrix(results_majority['confusion_matrix'], 
                      "Majority Class Baseline - Confusion Matrix")

print("\n💡 OBSERVATION:")
print("High overall accuracy (74%) but ZERO predictions for Left or Right!")
print("This model is useless for actual driving - it never turns.")
print("→ This is why we need to look at per-class F1 scores, not just accuracy.")

## 4. Baseline 2: Random Classifier

**What it does**: Randomly predicts Left/Forward/Right with equal probability

**Why test this?**
- Absolute worst-case baseline
- Expected accuracy: ~33% (random chance for 3 classes)
- Any real model should beat this easily

**If a model performs worse than random**: Something is seriously wrong with the implementation!

In [ ]:
# Create random classifier
# strategy='uniform' predicts each class with equal probability
random_clf = DummyClassifier(strategy='uniform', random_state=42)
random_clf.fit(X_train_flat, y_train)

# Predict
y_pred_random = random_clf.predict(X_test_flat)

# Evaluate
results_random = evaluate_model(y_test, y_pred_random, "Random Baseline")

# Plot confusion matrix
plot_confusion_matrix(results_random['confusion_matrix'],
                      "Random Baseline - Confusion Matrix")

print("\n💡 OBSERVATION:")
print(f"Random guessing achieves ~{results_random['accuracy']*100:.1f}% accuracy")
print("Any model below this is broken. Any model should beat this easily.")

## 5. Model 1: Logistic Regression

**What is Logistic Regression?**
- Linear classifier: finds a straight line (actually hyperplane) to separate classes
- Learns: `prediction = sign(w₁×pixel₁ + w₂×pixel₂ + ... + w₄₀₉₆×pixel₄₀₉₆ + bias)`
- Simple, fast, interpretable

**Expected performance** (based on EDA findings):
- PCA showed classes are NOT linearly separable
- Classes overlap completely in 2D projection
- **Prediction: 55-60% accuracy** (struggling with non-linear patterns)

**We'll test three scenarios**:
- A: Original imbalanced data
- B: Original data + class weights
- C: TFI-balanced data

### 5A: Logistic Regression - Original Data (No Balancing)

In [ ]:
# Train Logistic Regression on original imbalanced data
print("Training Logistic Regression on original data...")

start_time = time.time()
lr_original = LogisticRegression(
    max_iter=1000,           # Maximum iterations for convergence
    random_state=42,
    multi_class='multinomial',  # Use softmax for 3 classes
    solver='lbfgs'           # Optimization algorithm
)

lr_original.fit(X_train_flat, y_train)
train_time = time.time() - start_time

# Predict on test set
y_pred_lr_original = lr_original.predict(X_test_flat)

# Evaluate
results_lr_original = evaluate_model(y_test, y_pred_lr_original, 
                                     "Logistic Regression (No Balancing)")

print(f"\nTraining time: {train_time:.2f} seconds")

# Plot confusion matrix
plot_confusion_matrix(results_lr_original['confusion_matrix'],
                      "Logistic Regression (Original) - Confusion Matrix")

### 5B: Logistic Regression - Class Weights

In [ ]:
# Train Logistic Regression with class weights
# sklearn's class_weight='balanced' automatically computes weights
print("Training Logistic Regression with class weights...")

start_time = time.time()
lr_weighted = LogisticRegression(
    max_iter=1000,
    random_state=42,
    multi_class='multinomial',
    solver='lbfgs',
    class_weight='balanced'  # Use balanced class weights
)

lr_weighted.fit(X_train_flat, y_train)
train_time = time.time() - start_time

# Predict
y_pred_lr_weighted = lr_weighted.predict(X_test_flat)

# Evaluate
results_lr_weighted = evaluate_model(y_test, y_pred_lr_weighted,
                                     "Logistic Regression (Class Weights)")

print(f"\nTraining time: {train_time:.2f} seconds")

# Plot
plot_confusion_matrix(results_lr_weighted['confusion_matrix'],
                      "Logistic Regression (Weighted) - Confusion Matrix")

### 5C: Logistic Regression - TFI Balanced Data

In [ ]:
# Train Logistic Regression on TFI-balanced data
print("Training Logistic Regression on TFI-balanced data...")

start_time = time.time()
lr_tfi = LogisticRegression(
    max_iter=1000,
    random_state=42,
    multi_class='multinomial',
    solver='lbfgs'
)

lr_tfi.fit(X_train_tfi_flat, y_train_tfi)
train_time = time.time() - start_time

# Predict
y_pred_lr_tfi = lr_tfi.predict(X_test_flat)

# Evaluate
results_lr_tfi = evaluate_model(y_test, y_pred_lr_tfi,
                                "Logistic Regression (TFI Balanced)")

print(f"\nTraining time: {train_time:.2f} seconds")
print(f"Training set size: {len(X_train_tfi)} (vs {len(X_train)} original)")

# Plot
plot_confusion_matrix(results_lr_tfi['confusion_matrix'],
                      "Logistic Regression (TFI) - Confusion Matrix")

### Compare Logistic Regression Results

In [ ]:
# Create comparison table
import pandas as pd

lr_comparison = pd.DataFrame([
    {
        'Method': 'No Balancing',
        'Accuracy': f"{results_lr_original['accuracy']:.3f}",
        'F1-Macro': f"{results_lr_original['f1_macro']:.3f}",
        'F1-Left': f"{results_lr_original['f1_left']:.3f}",
        'F1-Forward': f"{results_lr_original['f1_forward']:.3f}",
        'F1-Right': f"{results_lr_original['f1_right']:.3f}"
    },
    {
        'Method': 'Class Weights',
        'Accuracy': f"{results_lr_weighted['accuracy']:.3f}",
        'F1-Macro': f"{results_lr_weighted['f1_macro']:.3f}",
        'F1-Left': f"{results_lr_weighted['f1_left']:.3f}",
        'F1-Forward': f"{results_lr_weighted['f1_forward']:.3f}",
        'F1-Right': f"{results_lr_weighted['f1_right']:.3f}"
    },
    {
        'Method': 'TFI Balanced',
        'Accuracy': f"{results_lr_tfi['accuracy']:.3f}",
        'F1-Macro': f"{results_lr_tfi['f1_macro']:.3f}",
        'F1-Left': f"{results_lr_tfi['f1_left']:.3f}",
        'F1-Forward': f"{results_lr_tfi['f1_forward']:.3f}",
        'F1-Right': f"{results_lr_tfi['f1_right']:.3f}"
    }
])

print("\n" + "="*80)
print("LOGISTIC REGRESSION: Comparison of Balancing Methods")
print("="*80)
print(lr_comparison.to_string(index=False))
print("="*80)

print("\n💡 KEY OBSERVATIONS:")
print("- Overall accuracy may DECREASE with balancing (that's okay!)")
print("- F1 scores for minority classes should INCREASE (that's what we want!)")
print("- F1-Macro gives equal weight to all classes (better metric than accuracy)")

## 6. Model 2: Random Forest

**What is Random Forest?**
- Ensemble of decision trees (collection of many trees voting)
- Each tree learns if-then rules: "if pixel[100] > 120 AND pixel[200] < 50, predict Left"
- Non-linear: can learn complex patterns
- Less prone to overfitting than single decision tree

**Expected performance** (based on EDA):
- Can learn some spatial patterns (better than Logistic Regression)
- **Prediction: 60-65% accuracy**
- Should handle non-linearity better (PCA showed classes overlap in linear space)

**Hyperparameters**:
- `n_estimators`: Number of trees (more = better, but slower)
- `max_depth`: Maximum tree depth (prevent overfitting)
- `min_samples_split`: Minimum samples to split a node

### 6A: Random Forest - Original Data

In [ ]:
# Train Random Forest on original data
print("Training Random Forest on original data...")

start_time = time.time()
rf_original = RandomForestClassifier(
    n_estimators=100,        # 100 trees
    max_depth=20,            # Limit depth to prevent overfitting
    min_samples_split=10,    # Need at least 10 samples to split
    random_state=42,
    n_jobs=-1                # Use all CPU cores
)

rf_original.fit(X_train_flat, y_train)
train_time = time.time() - start_time

# Predict
y_pred_rf_original = rf_original.predict(X_test_flat)

# Evaluate
results_rf_original = evaluate_model(y_test, y_pred_rf_original,
                                     "Random Forest (No Balancing)")

print(f"\nTraining time: {train_time:.2f} seconds")

# Plot
plot_confusion_matrix(results_rf_original['confusion_matrix'],
                      "Random Forest (Original) - Confusion Matrix")

### 6B: Random Forest - Class Weights

In [ ]:
# Train Random Forest with balanced class weights
print("Training Random Forest with class weights...")

start_time = time.time()
rf_weighted = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=10,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'  # Use balanced weights
)

rf_weighted.fit(X_train_flat, y_train)
train_time = time.time() - start_time

# Predict
y_pred_rf_weighted = rf_weighted.predict(X_test_flat)

# Evaluate
results_rf_weighted = evaluate_model(y_test, y_pred_rf_weighted,
                                     "Random Forest (Class Weights)")

print(f"\nTraining time: {train_time:.2f} seconds")

# Plot
plot_confusion_matrix(results_rf_weighted['confusion_matrix'],
                      "Random Forest (Weighted) - Confusion Matrix")

### 6C: Random Forest - TFI Balanced Data

In [ ]:
# Train Random Forest on TFI-balanced data
print("Training Random Forest on TFI-balanced data...")

start_time = time.time()
rf_tfi = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=10,
    random_state=42,
    n_jobs=-1
)

rf_tfi.fit(X_train_tfi_flat, y_train_tfi)
train_time = time.time() - start_time

# Predict
y_pred_rf_tfi = rf_tfi.predict(X_test_flat)

# Evaluate
results_rf_tfi = evaluate_model(y_test, y_pred_rf_tfi,
                                "Random Forest (TFI Balanced)")

print(f"\nTraining time: {train_time:.2f} seconds")
print(f"Training set size: {len(X_train_tfi)} (vs {len(X_train)} original)")

# Plot
plot_confusion_matrix(results_rf_tfi['confusion_matrix'],
                      "Random Forest (TFI) - Confusion Matrix")

### Compare Random Forest Results

In [ ]:
# Create comparison table
rf_comparison = pd.DataFrame([
    {
        'Method': 'No Balancing',
        'Accuracy': f"{results_rf_original['accuracy']:.3f}",
        'F1-Macro': f"{results_rf_original['f1_macro']:.3f}",
        'F1-Left': f"{results_rf_original['f1_left']:.3f}",
        'F1-Forward': f"{results_rf_original['f1_forward']:.3f}",
        'F1-Right': f"{results_rf_original['f1_right']:.3f}"
    },
    {
        'Method': 'Class Weights',
        'Accuracy': f"{results_rf_weighted['accuracy']:.3f}",
        'F1-Macro': f"{results_rf_weighted['f1_macro']:.3f}",
        'F1-Left': f"{results_rf_weighted['f1_left']:.3f}",
        'F1-Forward': f"{results_rf_weighted['f1_forward']:.3f}",
        'F1-Right': f"{results_rf_weighted['f1_right']:.3f}"
    },
    {
        'Method': 'TFI Balanced',
        'Accuracy': f"{results_rf_tfi['accuracy']:.3f}",
        'F1-Macro': f"{results_rf_tfi['f1_macro']:.3f}",
        'F1-Left': f"{results_rf_tfi['f1_left']:.3f}",
        'F1-Forward': f"{results_rf_tfi['f1_forward']:.3f}",
        'F1-Right': f"{results_rf_tfi['f1_right']:.3f}"
    }
])

print("\n" + "="*80)
print("RANDOM FOREST: Comparison of Balancing Methods")
print("="*80)
print(rf_comparison.to_string(index=False))
print("="*80)

## 7. Overall Comparison

Compare all baseline models to identify the best performer.

In [ ]:
# Create comprehensive comparison table
all_results = pd.DataFrame([
    {'Model': 'Majority Class', 'Balancing': 'N/A', 
     'Accuracy': f"{results_majority['accuracy']:.3f}",
     'F1-Macro': f"{results_majority['f1_macro']:.3f}",
     'F1-Left': f"{results_majority['f1_left']:.3f}",
     'F1-Right': f"{results_majority['f1_right']:.3f}"},
    
    {'Model': 'Random', 'Balancing': 'N/A',
     'Accuracy': f"{results_random['accuracy']:.3f}",
     'F1-Macro': f"{results_random['f1_macro']:.3f}",
     'F1-Left': f"{results_random['f1_left']:.3f}",
     'F1-Right': f"{results_random['f1_right']:.3f}"},
    
    {'Model': 'Logistic Regression', 'Balancing': 'None',
     'Accuracy': f"{results_lr_original['accuracy']:.3f}",
     'F1-Macro': f"{results_lr_original['f1_macro']:.3f}",
     'F1-Left': f"{results_lr_original['f1_left']:.3f}",
     'F1-Right': f"{results_lr_original['f1_right']:.3f}"},
    
    {'Model': 'Logistic Regression', 'Balancing': 'Class Weights',
     'Accuracy': f"{results_lr_weighted['accuracy']:.3f}",
     'F1-Macro': f"{results_lr_weighted['f1_macro']:.3f}",
     'F1-Left': f"{results_lr_weighted['f1_left']:.3f}",
     'F1-Right': f"{results_lr_weighted['f1_right']:.3f}"},
    
    {'Model': 'Logistic Regression', 'Balancing': 'TFI',
     'Accuracy': f"{results_lr_tfi['accuracy']:.3f}",
     'F1-Macro': f"{results_lr_tfi['f1_macro']:.3f}",
     'F1-Left': f"{results_lr_tfi['f1_left']:.3f}",
     'F1-Right': f"{results_lr_tfi['f1_right']:.3f}"},
    
    {'Model': 'Random Forest', 'Balancing': 'None',
     'Accuracy': f"{results_rf_original['accuracy']:.3f}",
     'F1-Macro': f"{results_rf_original['f1_macro']:.3f}",
     'F1-Left': f"{results_rf_original['f1_left']:.3f}",
     'F1-Right': f"{results_rf_original['f1_right']:.3f}"},
    
    {'Model': 'Random Forest', 'Balancing': 'Class Weights',
     'Accuracy': f"{results_rf_weighted['accuracy']:.3f}",
     'F1-Macro': f"{results_rf_weighted['f1_macro']:.3f}",
     'F1-Left': f"{results_rf_weighted['f1_left']:.3f}",
     'F1-Right': f"{results_rf_weighted['f1_right']:.3f}"},
    
    {'Model': 'Random Forest', 'Balancing': 'TFI',
     'Accuracy': f"{results_rf_tfi['accuracy']:.3f}",
     'F1-Macro': f"{results_rf_tfi['f1_macro']:.3f}",
     'F1-Left': f"{results_rf_tfi['f1_left']:.3f}",
     'F1-Right': f"{results_rf_tfi['f1_right']:.3f}"}
])

print("\n" + "="*90)
print("ALL BASELINE MODELS: Complete Comparison")
print("="*90)
print(all_results.to_string(index=False))
print("="*90)

In [ ]:
# Visualize F1 scores across models
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: F1-Macro comparison
models = ['Majority', 'Random', 'LR-Orig', 'LR-Weight', 'LR-TFI', 'RF-Orig', 'RF-Weight', 'RF-TFI']
f1_macros = [
    results_majority['f1_macro'],
    results_random['f1_macro'],
    results_lr_original['f1_macro'],
    results_lr_weighted['f1_macro'],
    results_lr_tfi['f1_macro'],
    results_rf_original['f1_macro'],
    results_rf_weighted['f1_macro'],
    results_rf_tfi['f1_macro']
]

axes[0].bar(range(len(models)), f1_macros, color='steelblue', alpha=0.7)
axes[0].set_xticks(range(len(models)))
axes[0].set_xticklabels(models, rotation=45, ha='right')
axes[0].set_ylabel('F1-Macro Score', fontsize=12)
axes[0].set_title('F1-Macro Comparison (Higher is Better)', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)
axes[0].set_ylim([0, 1])

# Plot 2: Per-class F1 for best models
# Compare best LR vs best RF
best_models = ['LR-Weight', 'LR-TFI', 'RF-Weight', 'RF-TFI']
best_results = [results_lr_weighted, results_lr_tfi, results_rf_weighted, results_rf_tfi]

x = np.arange(len(best_models))
width = 0.25

f1_left = [r['f1_left'] for r in best_results]
f1_forward = [r['f1_forward'] for r in best_results]
f1_right = [r['f1_right'] for r in best_results]

axes[1].bar(x - width, f1_left, width, label='Left', color='#FF6B6B', alpha=0.8)
axes[1].bar(x, f1_forward, width, label='Forward', color='#4ECDC4', alpha=0.8)
axes[1].bar(x + width, f1_right, width, label='Right', color='#45B7D1', alpha=0.8)

axes[1].set_xlabel('Model', fontsize=12)
axes[1].set_ylabel('F1 Score', fontsize=12)
axes[1].set_title('Per-Class F1 Scores', fontsize=14, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(best_models)
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

## 8. Feature Importance Analysis (Random Forest)

**What is feature importance?**
- Random Forest can tell us which pixels are most useful for predictions
- Higher importance = pixel is more useful for classification

**Why this is interesting**:
- Shows which parts of the image the model looks at
- Can visualize as a 64×64 heatmap
- Validates if model is learning sensible patterns (e.g., bottom edges for steering)

In [ ]:
# Get feature importances from best Random Forest model
# Use the TFI-balanced model as example
importances = rf_tfi.feature_importances_

# Reshape to image format (64×64)
importance_map = importances.reshape(64, 64)

# Plot as heatmap
plt.figure(figsize=(10, 8))
plt.imshow(importance_map, cmap='hot', interpolation='nearest')
plt.colorbar(label='Feature Importance')
plt.title('Random Forest Feature Importance Map\n(Which pixels matter most?)', 
          fontsize=14, fontweight='bold')
plt.xlabel('X coordinate', fontsize=12)
plt.ylabel('Y coordinate', fontsize=12)
plt.tight_layout()
plt.show()

print("💡 INTERPRETATION:")
print("Brighter areas = more important pixels for classification")
print("Expected: Bottom edges should be bright (used for steering decisions)")
print("If center is bright: Model learned to look at track position")

## 9. Summary and Key Findings

**Baseline models tested**:
1. Majority class baseline
2. Random baseline
3. Logistic Regression (linear)
4. Random Forest (non-linear)

**Class balancing methods compared**:
- None (original imbalanced data)
- Class weights
- TFI (Temporal Frame Interpolation)

In [ ]:
print("\n" + "="*90)
print("SUMMARY: Key Findings from Baseline Models")
print("="*90)

# Find best overall model
best_f1_macro = max([
    results_lr_original['f1_macro'],
    results_lr_weighted['f1_macro'],
    results_lr_tfi['f1_macro'],
    results_rf_original['f1_macro'],
    results_rf_weighted['f1_macro'],
    results_rf_tfi['f1_macro']
])

print("\n1. BASELINE PERFORMANCE BOUNDS:")
print(f"   • Worst case (Random):        {results_random['accuracy']:.1%} accuracy")
print(f"   • Naive baseline (Majority):  {results_majority['accuracy']:.1%} accuracy (but 0% on turns!)")
print(f"   • Best baseline F1-Macro:     {best_f1_macro:.3f}")

print("\n2. LOGISTIC REGRESSION FINDINGS:")
print(f"   • Original data:     {results_lr_original['accuracy']:.1%} accuracy")
print(f"   • With class weights: {results_lr_weighted['accuracy']:.1%} accuracy")
print(f"   • With TFI:          {results_lr_tfi['accuracy']:.1%} accuracy")
print("   → Confirms EDA prediction: Linear model struggles (PCA showed overlap)")
print("   → Class balancing helps minority classes but overall accuracy modest")

print("\n3. RANDOM FOREST FINDINGS:")
print(f"   • Original data:     {results_rf_original['accuracy']:.1%} accuracy")
print(f"   • With class weights: {results_rf_weighted['accuracy']:.1%} accuracy")
print(f"   • With TFI:          {results_rf_tfi['accuracy']:.1%} accuracy")
print("   → Non-linear model outperforms linear (as expected from PCA)")
print("   → Shows spatial patterns exist, but linear methods can't capture them")

print("\n4. CLASS BALANCING IMPACT:")
lr_right_improvement = results_lr_weighted['f1_right'] - results_lr_original['f1_right']
rf_right_improvement = results_rf_weighted['f1_right'] - results_rf_original['f1_right']
print(f"   • LR Right-class F1 improved by: {lr_right_improvement:.3f} with class weights")
print(f"   • RF Right-class F1 improved by: {rf_right_improvement:.3f} with class weights")
print("   ✅ Class balancing successfully improves minority class performance!")

print("\n5. WHAT THIS TELLS US ABOUT CNNs:")
print(f"   • Best baseline F1-Macro: {best_f1_macro:.3f}")
print("   • CNNs should beat this by learning spatial hierarchies")
print("   • If CNN < 0.65 F1-Macro: something is wrong (overfitting, bad architecture)")
print("   • If CNN > 0.75 F1-Macro: spatial features are being learned successfully")

print("\n" + "="*90)
print("RECOMMENDATIONS FOR NEXT STEPS")
print("="*90)
print("For CNN models (Notebook 04):")
print("  1. Use class weights OR TFI (whichever worked better here)")
print("  2. Target: F1-Macro > {:.2f} (beat best baseline)".format(best_f1_macro))
print("  3. Focus on Right class F1 (hardest to predict)")
print("  4. Use temporal splits only (no random split for final models)")
print("="*90)

## 10. Save Results

Save results for comparison with CNN models later.

In [ ]:
# Save all results to file
baseline_results = {
    'majority': results_majority,
    'random': results_random,
    'lr_original': results_lr_original,
    'lr_weighted': results_lr_weighted,
    'lr_tfi': results_lr_tfi,
    'rf_original': results_rf_original,
    'rf_weighted': results_rf_weighted,
    'rf_tfi': results_rf_tfi
}

# Convert numpy arrays to lists for JSON serialization
for key in baseline_results:
    if 'confusion_matrix' in baseline_results[key]:
        baseline_results[key]['confusion_matrix'] = baseline_results[key]['confusion_matrix'].tolist()

# Save as JSON
import json
with open('../results/baseline_results.json', 'w') as f:
    json.dump(baseline_results, f, indent=2)

print("✅ Results saved to: results/baseline_results.json")
print("\nThese results will be used for comparison in later notebooks.")

## Next Steps

**Notebook 04: CNN Models**
- Test convolutional neural networks
- Leverage spatial structure (what linear models missed)
- Expected: 65-75% accuracy, F1-Macro > 0.65
- Compare simple CNN vs deeper architectures

**Notebook 05: Temporal Models**  
- LSTM on frame sequences
- Address temporal lag problem
- Expected: Best performance (70-80% accuracy)

**Goal**: Beat the baseline F1-Macro of {:.3f}!".format(best_f1_macro)